# Building an Image Classifier Deep Network applying Transfer Learning

In [ ]:
import os

s3endpoint = os.environ['s3endpoint']
s3bucket = os.environ['s3bucket_data']
mount_path = os.environ['working_dir']
s3accessKey = os.environ['aws_access_key']
s3secretKey = os.environ['aws_secret_key']

In [ ]:
# import the required libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tf.keras.utils
from PIL import Image
import numpy as np
import os
import base64
from io import BytesIO

# Clean data in PVC

In [ ]:
# mount_path = "/data/"
# mount_path = ""
images_path = mount_path+"images"
models_path = mount_path+"models"

os.system("rm -rf "+images_path)
os.system("rm -rf "+models_path)
# os.system("ls "+images_path)




# Data Retrieval from S3

In [ ]:
import boto3
from botocore.client import Config
from PIL import Image

# s3endpoint='https://minio-api-test02.apps.rhods-internal.61tk.p1.openshiftapps.com',
# s3endpoint = "https://minio-api-demo-ai.apps.cluster-ssfg4.dynamic.opentlc.com"
# s3bucket = "data"

s3_client = boto3.client('s3',
        endpoint_url=s3endpoint,
        aws_access_key_id=s3accessKey,
        aws_secret_access_key=s3secretKey,
        # aws_access_key_id='ZpPJfDh82BVcKnCYdpKH',
        # aws_secret_access_key='cB6xKinAwdWxZh8gE3SwPKPPsBEnfakSMaipF1rz',
        config=Config(signature_version='s3v4'),
        region_name='us-east-1')

# Some demos might need 1000+ images to train the model
# The API 'list_objects_v2' limits results to 1000 keys, so we use pagination.
paginator = s3_client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=s3bucket)

# Image Validator (using PIL)
def is_valid_image_pillow(file_name):
    try:
        with Image.open(file_name) as img:
            img.verify()
            return True
    except (IOError, SyntaxError):
        return False

for page in pages:
    for obj in page['Contents']:
        # print(obj['Key'])
        if not obj['Key'].endswith("/"):
            print(obj['Key'])

            if not os.path.exists(os.path.dirname(mount_path+obj['Key'])):
                os.makedirs(os.path.dirname(mount_path+obj['Key']))
            s3_client.download_file(s3bucket,obj['Key'], mount_path+obj['Key'])

            # This code prevents corrupted images to break the training process
            if not is_valid_image_pillow(mount_path+obj['Key']):
                print("Deleting invalid image (maybe corrupt?): "+mount_path+obj['Key'])
                os.remove(mount_path+obj['Key'])